In [60]:
import os


In [3]:
os.chdir("../")

In [61]:
pwd

'd:\\projects\\MLflow_DL\\Machine-learning-project-with-MLflow'

In [62]:
from dataclasses import dataclass
from pathlib import Path

# this entity is for my return type, i.e how i want my config infor to be returned as
@dataclass(frozen=True)
class DataIngestionConfig():
    root_dir: Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path


In [68]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


print(CONFIG_FILE_PATH)
print(PARAMS_FILE_PATH)
print(SCHEMA_FILE_PATH)


config\config.yaml
params.yaml
schema.yaml


In [69]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.schema = read_yaml(schema_filepath)
            self.params = read_yaml(params_filepath)
            print(self.config)

            create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion
          
          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )
          return data_ingestion_config


In [71]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):


        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



# now updating the pipeline

In [72]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-22 15:14:17,983: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-22 15:14:17,984: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-22 15:14:17,986: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/maymunashah/Machine-learning-project-with-MLflow/main/wine+quality.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2025-01-22 15:14:17,988: INFO: common: created directory at: artifacts]
[2025-01-22 15:14:17,988: INFO: common: created directory at: artifacts/data_ingestion]


[2025-01-22 15:15:42,431: INFO: 2606948795: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 91353
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e09dc271395a544f6f655050dd47156058df5b4517da1bce195e95eb6e567889"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0E64:9F932:47154:490EA:6790FD8E
Accept-Ranges: bytes
Date: Wed, 22 Jan 2025 14:15:43 GMT
Via: 1.1 varnish
X-Served-By: cache-mxp6920-MXP
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1737555343.810087,VS0,VE223
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: fdc6484dfa3ee08c9165831dcbf4d72930a173cf
Expires: Wed, 22 Jan 2025 14:20:43 GMT
Source-Age: 0

]
